# Compute the LateX cutflow tables
- Will load the pkl files that contain the cutflows and the sumgenweight
- Will scale the events by the cross section
- Will save the yields in a dictionnary called ```cutflows -> Dict()```
- Will make the LateX table using the function ```make_composition_table()```

In [1]:
import glob
import json
import os
import pickle as pkl
import sys

import hist as hist2
import matplotlib.pyplot as plt
import mplhep as hep
import numpy as np
import pandas as pd
import pyarrow
import pyarrow.parquet as pq
import yaml
from scipy.special import softmax
from sklearn.metrics import auc, roc_curve

sys.path
sys.path.append("../python/")

import utils

plt.style.use(hep.style.CMS)
plt.rcParams.update({"font.size": 20})

pd.options.mode.chained_assignment = None

import glob
import os
import json
import pickle
import yaml
import math

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import pyarrow.parquet as pq
from sklearn.metrics import auc, roc_curve
from scipy.special import softmax

import hist as hist2
import matplotlib.pyplot as plt
import mplhep as hep

plt.style.use(hep.style.CMS)

import sys
sys.path
sys.path.append("../python/")

import utils

plt.rcParams.update({"font.size": 20})

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# get lumi
with open("../fileset/luminosity.json") as f:
    luminosity = json.load(f)
    
luminosity

{'ele': {'Run2': 137640.0,
  '2016APV': 19492.72,
  '2016': 16809.96,
  '2017': 41476.02,
  '2018': 59816.23},
 'mu': {'Run2': 137640.0,
  '2016APV': 19436.16,
  '2016': 16810.81,
  '2017': 41475.26,
  '2018': 59781.96},
 'lep': {'Run2': 137640.0,
  '2016APV': 19436.16,
  '2016': 16810.81,
  '2017': 41475.26,
  '2018': 59781.96},
 'had': {'Run2': 137640.0,
  '2016APV': 19436.16,
  '2016': 16810.81,
  '2017': 41475.26,
  '2018': 59781.96}}

In [4]:
def get_lumi(years, channels):
    lum_ = 0
    for year in years:
        lum = 0
        for ch in channels:
            lum += luminosity[ch][year] / 1000.0

        lum_ += lum / len(channels)    
    return lum_

# Read cutflows from pkl

In [5]:
def get_sum_sumgenweight(pkl_files, year, sample):
    """Load and sum the sumgenweight of each pkl file."""
    
    sum_sumgenweight = 0
    for ifile in pkl_files:
        with open(ifile, "rb") as f:
            metadata = pkl.load(f)            
        sum_sumgenweight = sum_sumgenweight + metadata[sample][year]["sumgenweight"]

    return sum_sumgenweight


def get_xsecweight(pkl_files, year, ch, sample, is_data):
    
    if not is_data:
        # find xsection
        f = open("../fileset/xsec_pfnano.json")
        xsec = json.load(f)
        f.close()
        try:
            xsec = eval(str((xsec[sample])))
        except ValueError:
            print(f"sample {sample} doesn't have xsecs defined in xsec_pfnano.json so will skip it")
            return None

        # get overall weighting of events.. each event has a genweight...
        # sumgenweight sums over events in a chunk... sum_sumgenweight sums over chunks
        xsec_weight = (xsec * luminosity[ch][year]) / get_sum_sumgenweight(pkl_files, year, sample)
    else:
        xsec_weight = 1
    return xsec_weight

def get_cutflow(pkl_files, year, ch, sample, is_data):
    """
    Get cutflow from metadata but multiply by xsec-weight
    """
    xsec_weight = get_xsecweight(pkl_files, year, ch, sample, is_data)
        
    cuts = [
        "sumgenweight",
        "Trigger",
        "METFilters",
        "OneLep",
        "NoTaus",
        "AtLeastOneFatJet",
        "CandidateJetpT",
        "LepInJet",
        "JetLepOverlap",
        "dPhiJetMET",
        "MET",
    ]
        
    if year == "2018":
        cuts += ["HEMCleaning"]
        
    evyield = dict.fromkeys(cuts, 0)
    for ik, pkl_file in enumerate(pkl_files):
        with open(pkl_file, "rb") as f:
            metadata = pkl.load(f)
            
        cutflows = metadata[sample][year]["cutflows"][ch]

        for key in evyield.keys():

            if key == "sumgenweight":
                evyield[key] += metadata[sample][year][key] * xsec_weight
            else:
                evyield[key] += cutflows[key] * xsec_weight        
    return evyield

# Adding a cut from the parquets

In [59]:
### This is your configuration. specefy which channels, years, samples, and directory of pkl files to use.
channels = [
    "ele", 
    "mu",
]
years = [
#     "2018", 
    "2017",
#     "2016", 
#     "2016APV",
]

samples = [
    "ggF", 
    "VBF",  
    "WH",
    "ZH",    
    "ttH",
    "WJetsLNu",
    "TTbar",
    "SingleTop",
    "Diboson",
    "EWKvjets",
    "DYJets",
    "WZQQ",
    "Data",
]

samples_dir = {
#     "2016":    "../eos/Oct10_hww_2016",
#     "2016APV": "../eos/Oct10_hww_2016APV",    
#     "2017":    "../eos/Oct10_hww_2017",    
#     "2018":    "../eos/Oct10_hww_2018",
    
    "2016":    "../eos/hww/Dec20_hww_2016",
    "2016APV": "../eos/hww/Dec20_hww_2016APV",    
    "2017":    "../eos/hww/Dec20_hww_2017",
#     "2017":    "../eos/hww/Jun5_hww_2017",
    "2018":    "../eos/hww/Dec20_hww_2018",    
}

In [60]:
cutflows = {}
for year in years:
#     if year != "2016":
#         continue
    print(f"Processing year {year}")
    
    cutflows[year] = {}
    
    for ch in channels:
#         if ch != "mu": 
#             continue
        
        print(f"  {ch} channel")
        cutflows[year][ch] = {}

        condor_dir = os.listdir(samples_dir[year])

        for sample in condor_dir:

            # first: check if the sample is in one of combine_samples_by_name
            sample_to_use = None
            for key in utils.combine_samples_by_name:
                if key in sample:
                    sample_to_use = utils.combine_samples_by_name[key]
                    break

            # second: if not, combine under common label
            if sample_to_use is None:
                for key in utils.combine_samples:
                    if key in sample:
                        sample_to_use = utils.combine_samples[key]
                        break
                    else:
                        sample_to_use = sample

            if sample_to_use not in samples:
                continue

            is_data = False
            if sample_to_use == "Data":
                is_data = True

            out_files = f"{samples_dir[year]}/{sample}/outfiles/"
            pkl_files = glob.glob(f"{out_files}/*.pkl")

            if len(pkl_files) == 0:
                continue

            parquet_files = glob.glob(f"{out_files}/*_{ch}.parquet")
            
            try:
                data = pd.read_parquet(parquet_files)
            except pyarrow.lib.ArrowInvalid:
                # empty parquet because no event passed selection
#                 print(f"No parquet file for {sample}")
                continue

            if len(data) == 0:
#                 print(f"Hi, No parquet file for {sample}")
                continue
    
            if sample_to_use not in cutflows[year][ch].keys():
                cutflows[year][ch][sample_to_use] = get_cutflow(pkl_files, year, ch, sample, is_data)
            else:
                temp = get_cutflow(pkl_files, year, ch, sample, is_data)
                for key in cutflows[year][ch][sample_to_use]:
                    cutflows[year][ch][sample_to_use][key] += temp[key]
            
    print(f"------------------------------------------")

Processing year 2017
  ele channel
  mu channel
------------------------------------------


In [61]:
samples = cutflows["2017"]["ele"].keys()  # samples
samples

dict_keys(['WJetsLNu', 'EWKvjets', 'WH', 'TTbar', 'SingleTop', 'ggF', 'DYJets', 'Data', 'Diboson', 'WZQQ', 'ttH', 'VBF', 'ZH'])

In [62]:
from make_stacked_hists import make_events_dict

presel = {
        "mu": {
#             "fj_mass": "fj_mass>40",
#             "THWW>0.75": "THWW>0.750",
        },
        "ele": {
#             "fj_mass": "fj_mass>40",
#             "THWW>0.75": "THWW>0.750",
        },
}

THWW_path = "../../weaver-core-dev/experiments_finetuning/v35_30/model.onnx"

events_dict = make_events_dict(years, channels, samples_dir, samples, presel, THWW_path)

INFO:root:Processing 2017 ele channel
INFO:root:Processing 2017 mu channel


# Add the cut to the cutflow dict

In [63]:
presel = {
        "mu": {
            "fj_mass": "fj_mass>40",
            "THWW>0.75": "fj_mass>40 & THWW>0.75",
        },
        "ele": {
            "fj_mass": "fj_mass>40",
            "THWW>0.75": "fj_mass>40 & THWW>0.75",
        },
}

In [64]:
for ch in channels:
    for cut, sel in list(presel[ch].items()):
        for sample in samples:
            for year in years:

                df = events_dict[year][ch][sample]
                df = df.query(sel)
                
                w = df["nominal"]

                cutflows[year][ch][sample][cut] = w.sum()

In [65]:
cutflows["2017"]["mu"]["WJetsLNu"]     # take a quick look

{'sumgenweight': 68395907.29737747,
 'Trigger': 10561522.711528933,
 'METFilters': 10555817.741991894,
 'OneLep': 10519172.344996287,
 'NoTaus': 8947622.779191008,
 'AtLeastOneFatJet': 1028856.0673407008,
 'CandidateJetpT': 467053.5136689998,
 'LepInJet': 168386.5512486023,
 'JetLepOverlap': 64711.53408324735,
 'dPhiJetMET': 45520.409841926725,
 'MET': 42561.39834191103,
 'fj_mass': 48846.88393434201,
 'THWW>0.75': 1352.2131225216735}

In [66]:
cutflows["2017"]["ele"]["WJetsLNu"]     # take a quick look

{'sumgenweight': 68397160.59608002,
 'Trigger': 8685086.17951225,
 'METFilters': 8680478.334096672,
 'OneLep': 6799301.343566881,
 'NoTaus': 6799301.343566881,
 'AtLeastOneFatJet': 840620.0699936592,
 'CandidateJetpT': 379483.6950056169,
 'LepInJet': 148978.8602961507,
 'JetLepOverlap': 50174.49584002223,
 'dPhiJetMET': 34600.7690071595,
 'MET': 32258.45996483975,
 'fj_mass': 37709.64440129174,
 'THWW>0.75': 970.0300195818509}

# Combine different channels

In [44]:
common_cuts = cutflows["2018"]["mu"]["WJetsLNu"]
common_cuts

{'sumgenweight': 68395907.29737747,
 'Trigger': 10561522.711528933,
 'METFilters': 10555817.741991894,
 'OneLep': 10519172.344996287,
 'NoTaus': 8947622.779191008,
 'AtLeastOneFatJet': 1028856.0673407008,
 'CandidateJetpT': 467053.5136689998,
 'LepInJet': 168386.5512486023,
 'JetLepOverlap': 64711.53408324735,
 'dPhiJetMET': 45520.409841926725,
 'MET': 42561.39834191103,
 'fj_mass': 48649.57380277161,
 'THWW>0.75': 1347.2342165088326}

In [67]:
def combine_channels(cutflows):

    # combine both channels
    cutflows_new = {}
    for year in cutflows.keys():
        cutflows_new[year] = {}
        cutflows_new[year]["lep"] = {}
        
        for ch in ["mu", "ele"]:
            for sample in cutflows[year][ch]:
                                
                if sample not in cutflows_new[year]["lep"]:
                    cutflows_new[year]["lep"][sample] = {}
                
                for cut in cutflows[year][ch][sample]:
                    
                    if (year != "2018") and (cut == "HEMCleaning"):
                        continue
                    
                    if cut not in cutflows_new[year]["lep"][sample]:
                        cutflows_new[year]["lep"][sample][cut] = cutflows[year][ch][sample][cut]
                    else:
                        cutflows_new[year]["lep"][sample][cut] += cutflows[year][ch][sample][cut]
        cutflows[year] = {**cutflows[year], **cutflows_new[year]}
        
    return cutflows

In [68]:
cutflows = combine_channels(cutflows)

In [69]:
cutflows["2018"].keys()

KeyError: '2018'

In [ ]:
cutflows["2018"]["ele"]["WJetsLNu"]

In [ ]:
cutflows["2018"]["mu"]["WJetsLNu"]

In [ ]:
cutflows["2018"]["lep"]["WJetsLNu"]

# Combine different years

In [70]:
def combine_years(cutflows):
    """Will remove the HEM cleaning cutflow from 2018 first."""
    
    whatever_year = list(cutflows.keys())[0]
    channels = cutflows[whatever_year].keys()
    
    # combine all years
    cutflows_new = {}
    cutflows_new["Run2"] = {}
    
    for ch in channels:
        cutflows_new["Run2"][ch] = {}
        
        for year in cutflows:
            for sample in cutflows[year][ch]:
                
                if sample not in cutflows_new["Run2"][ch]:
                    cutflows_new["Run2"][ch][sample] = {}

                for cut in cutflows[year][ch][sample]:
                    if "HEM" in cut:
                        continue
                    if cut not in cutflows_new["Run2"][ch][sample]:
                        cutflows_new["Run2"][ch][sample][cut] = cutflows[year][ch][sample][cut]
                    else:
                        cutflows_new["Run2"][ch][sample][cut] += cutflows[year][ch][sample][cut]

    cutflows = {**cutflows, **cutflows_new}

    return cutflows

In [71]:
cutflows = combine_years(cutflows)

In [72]:
cutflows["2017"]["ele"].keys()

dict_keys(['WJetsLNu', 'EWKvjets', 'WH', 'TTbar', 'SingleTop', 'ggF', 'DYJets', 'Data', 'Diboson', 'WZQQ', 'ttH', 'VBF', 'ZH'])

In [73]:
cutflows.keys()

dict_keys(['2017', 'Run2'])

In [51]:
cutflows["Run2"].keys()

dict_keys(['ele', 'mu', 'lep'])

# Combine non-dominant backgrounds

In [74]:
# combine non-dominant backgrounds under others
dominant_bkgs = ["WJetsLNu", "TTbar"]
signals = ["ggF", "VBF", "WH", "ZH", "ttH"]

for year in cutflows:
    for ch in cutflows[year]:
        cutflows[year][ch]["Others"] = dict.fromkeys(cutflows[year][ch]["WJetsLNu"], 0)
        for sample in cutflows[year][ch]:
            if sample == "Data":
                continue
            if sample not in signals+dominant_bkgs:
                for cut in cutflows[year][ch][sample]:
                    cutflows[year][ch]["Others"][cut] += cutflows[year][ch][sample][cut]

In [75]:
cutflows["2017"]["ele"].keys()

dict_keys(['WJetsLNu', 'EWKvjets', 'WH', 'TTbar', 'SingleTop', 'ggF', 'DYJets', 'Data', 'Diboson', 'WZQQ', 'ttH', 'VBF', 'ZH', 'Others'])

In [76]:
cutflows["2017"]["lep"]["Others"]

{'sumgenweight': 504923757.8414528,
 'Trigger': 70230387.808001,
 'METFilters': 70200411.81948991,
 'OneLep': 46194136.59237416,
 'NoTaus': 42957715.67864163,
 'AtLeastOneFatJet': 1239221.4055518333,
 'CandidateJetpT': 570697.1902661477,
 'LepInJet': 280836.05979165446,
 'JetLepOverlap': 106246.29007646533,
 'dPhiJetMET': 63896.01737911475,
 'MET': 55330.25217421119,
 'fj_mass': 42631.21810229422,
 'THWW>0.75': 753.6171800231882}

# LateX cutflow table

In [77]:
cuts = [
    "sumgenweight",
    "Trigger",
    "METFilters",
    "OneLep",        
    "NoTaus",
    "AtLeastOneFatJet",
    "CandidateJetpT",
    "LepInJet",
    "JetLepOverlap",
    "dPhiJetMET",
    "MET",
    "HEMCleaning",
]

for cut in presel["mu"]:
    cuts += [cut]

In [78]:
cut_to_label = {
    "sumgenweight": "sumgenweight",        
    "HEMCleaning": "HEMCleaning",    
    "Trigger": "Trigger",
    "METFilters": "METFilters",
    "OneLep": "n Leptons = 1",
    "NoTaus": "n Taus = 0",
    "AtLeastOneFatJet": r"n FatJets $>=$ 1",
    "CandidateJetpT": r"j $p_T > 250$GeV",
    "LepInJet": r"$\Delta R(j, \ell) < 0.8$",
    "JetLepOverlap": r"$\Delta R(j, \ell) > 0.03$",
    "dPhiJetMET": r"$\Delta \phi(\mathrm{MET}, j)<1.57$",
    "MET": r"$\mathrm{MET}>20$",
    
    "None": "None",

    "fj_mass": r"j $\mathrm{softdrop} > 40$GeV",
    
    "THWW>0.75": r"$\ensuremath{T_{\text{HWW}}^{\ell\nu qq}} > 0.75$",
} 


In [79]:
parquet_to_latex = {
    "WJetsLNu": "$\PW(\Pell\PGn)$+",
    "TTbar": "\\ttbar",
    "Others": "Other MC",

    "ggF": "ggF",
    "VBF": "VBF",
    "WH": "WH",
    "ZH": "ZH",    
    "ttH": "$t\\bar{t}H$",    
    
    "Data": "Data",
}

def make_latex_cutflow_table(cutflows_dict, year, ch, add_data=False, add_sumgenweight=False):
    """Will use the cutflows dictionary to make the LateX table we have in the AN."""
    
    samples_bkg = ["WJetsLNu", "TTbar", "Others"]
    samples_sig = ["ggF","VBF", "WH", "ZH", "ttH"]

    ### backgrounds
    headers = [parquet_to_latex[s] for s in samples_bkg]
    
    textabular = f"l{'r'*len(headers)}"
    textabular += "|r"
    
    texheader = "\\textbf{Inclusive Selection}" + " & " + " & ".join(headers) + " & Total MC "
    if add_data:
        textabular += "|r"
        texheader += "& Data "
    texheader += "\\\\"
    texdata = "\\hline\n"
    
    data = dict()
    
    for cut in cuts: 
        if (year != "2018") and (cut == "HEMCleaning"):
            continue
            
        if not add_sumgenweight and cut == "sumgenweight":
            continue
    
        data[cut] = []

        for sample in samples_bkg:            
            data[cut].append(round(cutflows_dict[year][ch][sample][cut]))
            
        totalmc = 0
        for sample in (samples_bkg + samples_sig):
            totalmc += round(cutflows_dict[year][ch][sample][cut])
            
        data[cut].append(totalmc)
        
        if add_data:
            data[cut].append(round(cutflows_dict[year][ch]["Data"][cut]))

    for label in data:
        if label == "z":
            texdata += "\\hline\n"
        texdata += f"{cut_to_label[label]} & {' & '.join(map(str,data[label]))} \\\\\n"
        
    texdata += "\\hline\n"    

    ### signal
    headers2 = [parquet_to_latex[s] for s in samples_sig]
    texheader2 = " & " + " & ".join(headers2) + "\\\\"
    texdata2 = "\\hline\n"

    textabular2 = f"l{'r'*len(headers2)}"
    
    data = dict()
    for cut in cuts:
        if (year != "2018") and (cut == "HEMCleaning"):
            continue
            
        data[cut] = []

        for sample in samples_sig:
            data[cut].append(round(cutflows_dict[year][ch][sample][cut]))
        
    for label in data:
        if label == "z":
            texdata += "\\hline\n"
        texdata2 += f"{cut_to_label[label]} & {' & '.join(map(str,data[label]))} \\\\\n"    

    # make table
    print("\\begin{table}[!htp]")
    print("\\begin{center}")
    
    print("\\begin{tabular}{"+textabular+"}")
    print(texheader)
    print(texdata,end="")
    print("\\end{tabular}")

    print("\\begin{tabular}{"+textabular2+"}")
    print(texheader2)
    print(texdata2,end="")
    print("\\end{tabular}")
    
    
    if ch == "lep":
        print("\\caption{Event yield of " + year + " Monte Carlo samples normalized to " + str(round(get_lumi([year], [ch]))) + "\\fbinv.}")        
    else:
        print("\\caption{Event yield of " + ch + " channel " + year + " Monte Carlo samples normalized to " + str(round(get_lumi([year], [ch]))) + "\\fbinv.}")
        
    print("\\label{sel-tab-cutflow" + year + "}")
    print("\\end{center}")
    print("\\end{table}")    

In [80]:
make_latex_cutflow_table(cutflows, "2017", "lep", add_data=True, add_sumgenweight=True)

\begin{table}[!htp]
\begin{center}
\begin{tabular}{lrrr|r|r}
\textbf{Inclusive Selection} & $\PW(\Pell\PGn)$+ & \ttbar & Other MC & Total MC & Data \\
\hline
sumgenweight & 136793068 & 68995474 & 504923758 & 711218464 & 1202391493 \\
Trigger & 19246609 & 6978344 & 70230388 & 96493182 & 619791460 \\
METFilters & 19236296 & 6973776 & 70200412 & 96448311 & 619630992 \\
n Leptons = 1 & 17318474 & 5893155 & 46194137 & 69438252 & 442103860 \\
n Taus = 0 & 15746924 & 4704050 & 42957716 & 63437606 & 424640213 \\
n FatJets $>=$ 1 & 1869476 & 882238 & 1239221 & 3993345 & 5032388 \\
j $p_T > 250$GeV & 846537 & 448262 & 570697 & 1866770 & 2000795 \\
$\Delta R(j, \ell) < 0.8$ & 317365 & 198780 & 280836 & 797719 & 993318 \\
$\Delta R(j, \ell) > 0.03$ & 114886 & 171166 & 106246 & 392979 & 456364 \\
$\Delta \phi(\mathrm{MET}, j)<1.57$ & 80121 & 107940 & 63896 & 252418 & 252747 \\
$\mathrm{MET}>20$ & 74820 & 101745 & 55330 & 232323 & 229103 \\
j $\mathrm{softdrop} > 40$GeV & 86557 & 81306 & 42631 & 210